In [1]:
board = ['euttel', 'cnorke', 'bckluy', 'saceth', 'maipct', 'andwio', 'ynnsma', 'oaieto'] 
board = [list(x) for x in board]
board

[['e', 'u', 't', 't', 'e', 'l'],
 ['c', 'n', 'o', 'r', 'k', 'e'],
 ['b', 'c', 'k', 'l', 'u', 'y'],
 ['s', 'a', 'c', 'e', 't', 'h'],
 ['m', 'a', 'i', 'p', 'c', 't'],
 ['a', 'n', 'd', 'w', 'i', 'o'],
 ['y', 'n', 'n', 's', 'm', 'a'],
 ['o', 'a', 'i', 'e', 't', 'o']]

In [2]:
!curl -O https://raw.githubusercontent.com/dolph/dictionary/master/enable1.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1702k  100 1702k    0     0  2414k      0 --:--:-- --:--:-- --:--:-- 2453k


In [3]:
!curl -O https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75153  100 75153    0     0   501k      0 --:--:-- --:--:-- --:--:--  535k


In [2]:
with open('enable1.txt') as f:
  words=set(f.read().splitlines())

In [3]:
with open('google-10000-english-no-swears.txt') as f:
    top_thousand=set(f.read().splitlines()[:4000])
    top_ten_thousand=[x for x in f.read().splitlines() if len(x)>3]

In [4]:
word_beginings = set()
for word in words:
  for i in range(len(word)+1):
      word_beginings.add(word[:i])

top_thousand_beginings = set()
for word in top_thousand:
  for i in range(len(word)+1):
      top_thousand_beginings.add(word[:i])

In [13]:
def in_bounds(coord):
  x,y = coord
  return 0 <= x < 6 and 0 <= y < 8

def get_neighbors(coord):
  x,y = coord
  return [out for out in [(x,y+1),(x,y-1),(x-1,y),(x-1,y+1),(x-1,y-1),(x+1,y),(x+1,y+1),(x+1,y-1)] if in_bounds(out)]

def get_adjacent(coord):
  x,y = coord
  return [out for out in [(x,y+1),(x,y-1),(x-1,y),(x+1,y)] if in_bounds(out)]

def get_diagonal(coord):
  x,y = coord
  return [out for out in [(x-1,y+1),(x-1,y-1),(x+1,y+1),(x+1,y-1)] if in_bounds(out)]

def allowed_spangram_part(letters):
    if letters in word_beginings:
        return True
    for i in range(len(letters)):
        if letters[:i] in top_thousand and letters[i:] in top_thousand_beginings:
            return True
    return False

def allowed_spangram(letters):
    if letters in words:
        return True
    for i in range(len(letters)):
        if letters[:i] in top_thousand and letters[i:] in top_thousand:
            return True
    return False

def get_poss_words(coords, letters):
  if letters not in word_beginings:
    return []
  running = []
  if len(letters) >= 4 and letters in words:
    running.append((letters, coords))

  poss = [x for x in get_neighbors(coords[-1]) if x not in coords]
  out = []
  for c in poss:
    out += get_poss_words(coords+[c], letters+board[c[1]][c[0]])
  return running+out

def get_poss_words_edge(coords, letters, edges):
  if not allowed_spangram_part(letters):
    return []
  running = []
  if len(letters) >= 4 and allowed_spangram(letters) and coords[-1] in edges:
    running.append((letters, coords))
  poss = [x for x in get_neighbors(coords[-1]) if x not in coords]
  out = []
  for c in poss:
    out += get_poss_words_edge(coords+[c], letters+board[c[1]][c[0]], edges)
  return running+out

edge0 = [(0,y) for y in range(8)]
edge1 = [(5,y) for y in range(8)]
edge2 = [(x,0) for x in range(6)]
edge3 = [(x,7) for x in range(6)]

spangrams = []

for start in edge0:
  spangrams += get_poss_words_edge([start], board[start[1]][start[0]], edge1)
for start in edge1:
  spangrams += get_poss_words_edge([start], board[start[1]][start[0]], edge0)
for start in edge2:
  spangrams += get_poss_words_edge([start], board[start[1]][start[0]], edge3)
for start in edge3:
  spangrams += get_poss_words_edge([start], board[start[1]][start[0]], edge2)

found_words = []

for x in range(6):
  for y in range(8):
    found_words += get_poss_words([(x,y)], board[y][x])

found_words_old = found_words
found_words = []
fws = set()
for word in found_words_old:
    if word[0] not in fws:
        fws.add(word[0])
        found_words.append(word)

def get_not_overlapping(already_visted, pos):
    out = []
    for word in pos:
        if len(set(already_visted+word[1])) == len(already_visted)+len(word[1]):
            out.append(word)
    return out

def diagonal_line_crossed(boundry, prev, pos):
    t1 = (pos[0], prev[1])
    t2 = (prev[0], pos[1])
    for i in range(len(boundry)-1):
        if boundry[i] == t1 and boundry[i+1] == t2:
            return True
        if boundry[i] == t2 and boundry[i+1] == t1:
            return True
    return False
    
def flood_fill(boundry, start):
  out = [start]
  to_check = [start]
  while to_check:
    top = to_check[-1]
    to_check.pop(-1)
    for n in get_adjacent(top):
      if n in out or n in boundry:
        continue
      to_check.append(n)
      out.append(n)
    for n in get_diagonal(top):
      if n in out or n in boundry or diagonal_line_crossed(boundry, top, n):
        continue
      to_check.append(n)
      out.append(n)
  return out

mem = set()
    
def dfs(words, already_visited, pos):
  if tuple(sorted(words)) in mem:
      return []
  pos = get_not_overlapping(already_visited, pos)
  output = [words]
  for word, coords in pos:
      output += dfs(words + (word,), already_visited+coords, pos)
  mem.add(tuple(sorted(words)))
  return output

def tidy(x):
    t = [tuple(sorted(a[1:])) for a in x]
    t = list(set(t))
    t.sort(key = lambda p: sum([top_ten_thousand.index(k) if k in top_ten_thousand else 20000 for k in p]))
    return t

def num_to_edge(i):
    i = i % 24
    if i<5:
        return (i,0)
    elif i<12:
        return (5,i-5)
    elif i<17:
        return (17-i,7)
    else:
        return (0,24-i)

spangrams.sort(reverse=True, key=lambda x: (len(x[0]) + (4 if x[0] in words else 0)))

spangrams = [x for x in spangrams if len(set(list(x[1])+edge0+edge1+edge2+edge3)) + 4 >= len(x[1])+2*(6+8)-4]

for sp in spangrams[:20]:
    mx = 0 
    outp = []
    print(sp[0])

    pc0 = 0
    pc1 = 12

    while num_to_edge(pc0) in sp[1]:
        pc0 += 1

    while pc1<13+24:
        pc1 += 1
        if num_to_edge(pc1) in sp[1]:
            continue
        if len(set(flood_fill(sp[1], num_to_edge(pc0)) + flood_fill(sp[1], num_to_edge(pc1)) + sp[1])) < 48:
            continue
        break
        
    if pc1 == 13+24:
        continue
                
    if len(flood_fill(sp[1], num_to_edge(pc0))) <= len(flood_fill(sp[1], num_to_edge(pc1))):
        corner0 = num_to_edge(pc0)
        corner1 = num_to_edge(pc1)
    else:
        corner0 = num_to_edge(pc1)
        corner1 = num_to_edge(pc0)
    
    mem = set()
    for pos in dfs((sp[0],), sp[1], get_not_overlapping(flood_fill(sp[1], corner0), found_words)):
        if sum([len(x) for x in pos]) > mx:
            mx = sum([len(x) for x in pos])
            outp = [pos]
        if sum([len(x) for x in pos]) == mx:
            outp.append(pos)
    
    mem = set()
    if mx-len(sp[1]) == len(flood_fill(sp[1], corner1)):
        mx2 = 0 
        outp2 = []
        for pos in dfs((sp[0],), sp[1], get_not_overlapping(flood_fill(sp[1], corner1), found_words)):
            if sum([len(x) for x in pos]) > mx2:
                mx2= sum([len(x) for x in pos])
                outp2 = [pos]
            if sum([len(x) for x in pos]) == mx2:
                outp2.append(pos)
        if mx+mx2-len(sp[1]) == 48:
            print((sp[0],) + tidy(outp)[0] + tidy(outp2)[0])

sandwich
('sandwich', 'bacon', 'lettuce', 'pickle', 'turkey', 'mayonnaise', 'tomato')
timesindiana
timesindiana
indiaaccount
indiaaccount
indiaaccount


KeyboardInterrupt: 